# Auto Correct For Non-word Errors
### Using Levenshtein Edit Distance

### Import libraries

In [4]:
import re
import os, sys
import numpy as np
import pandas as pd
from collections import Counter


### Create Function to Load and Preprocess the Text Corpus

In [5]:
def process_data(file_name):
    """
    Process a text corpus by lowercasing, tokenizing by words.

    Params:
    ----------
    file_name: str
        The text corpus to process.

    Returns:
    ----------
    word_tokens: list
        List of words extracted from the corpus in lowercase 

    """
    # load in the text corpus
    with open(file_name, 'r') as file:
        corpus = file.read()

    # Change text to lower case
    corpus_lower = corpus.lower()

    # Extract only words having letters two or more
    word_tokens = re.findall(r'\w\w+', corpus_lower)
    
    return word_tokens

### Data Preprocessing

In [6]:
# word_tokens = process_data('./dataset/shakespeare.txt')

word_tokens = []
for file in os.listdir('./datasets/coca-samples-text/'):
    if file.endswith('.txt'):
        filename = os.path.join(os.getcwd(), 'datasets', 'coca-samples-text', file)
        word_tokens += process_data(file_name=filename)

# # create the vocabulary from the dataset
vocab = set(word_tokens)
print('Length of vocab: ', len(vocab))


Length of vocab:  136840


### Create Function to generate Word Count Dictionary

In [7]:
def get_count(word_tokens):
    """ 
    Create a dictionary where the keys are the unique words and values are occurences of those words in the given list of words.

    Params:
    ----------
    word_tokens: list
        List of words
    
    Returns:
    ----------
    word_count: dict
        A dictionary where the keys are the unique words and values are occurences of those words in the given list of words.
    
    """

    word_count = Counter(word_tokens)
    return word_count 



In [8]:
word_count = get_count(word_tokens=word_tokens)
print('First 10 entries in the word count dictionary:\n', list(word_count.items())[:10])

First 10 entries in the word count dictionary:
 [('4000241', 1), ('think', 13889), ('it', 104072), ('is', 94201), ('safe', 1006), ('to', 238874), ('say', 9063), ('that', 124059), ('ours', 155), ('the', 462883)]


### Compute the Word Probabilites from the Word Count dictionary

$$P(w_i) = \frac{C(w_i)}{M}$$
where,

$C(w_i)$ is the total number of times $w_i$ appears in the corpus.

$M$ is the total number of words in the corpus.

In [9]:
def word_probs(word_count):
    """
    Compute the word probabilities from the given word count dictionary.

    Params:
    ----------
    word_count: dict
        A dictionary where the keys are the unique words and values are occurences of those words in the given list of words.
    
    Returns:
    ----------
    word_proba: dict
        A dictionary where the keys are the unique words and values are probabilities of those words in the given list of words.

    """
    # total number of words in the corpus
    M = sum(word_count.values())
    word_proba = { key: count/M for key, count in word_count.items()}

    return word_proba


In [10]:
# check the word probabilities dictionary
word_proba = word_probs(word_count=word_count)
list(word_proba.items())[:5]

[('4000241', 1.136029411347048e-07),
 ('think', 0.001577831249419915),
 ('it', 0.011822885289770999),
 ('is', 0.010701510657830328),
 ('safe', 0.00011428455878151304)]

We will implement below funtions:
* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 

In [11]:
def delete_letter(word):
    """
    Given a word, it returns all the possible strings that have one character removed.

    Params:
    ----------
    word: str
        The word to process
    
    Returns:
    ----------
    word_list: list
        List of possible strings.
    """
    word_list = list(set([ word[:i] + word[i+1:] for i in range(len(word)) ]))
    return word_list



In [12]:
def insert_letter(word):
    """
    Given a word, it returns all the possible strings that have an additional character inserted.

    Params:
    ----------
    word: str
        The word to process
    
    Returns:
    ----------
    word_list: list
        List of possible strings.
    """

    alphabets = 'abcdefghijklmnopqrstuvwxyz'
    word_list = list(set([ word[:i] + char + word[i:] for char in alphabets for i in range(len(word)+1)  ]))
    return word_list


In [13]:
def replace_letter(word):
    """
    Given a word, it returns all the possible strings that have one character replaced by another different letter.

    Params:
    ----------
    word: str
        The word to process
    
    Returns:
    ----------
    word_list: list
        List of possible strings.
    """
    alphabets = 'abcdefghijklmnopqrstuvwxyz'    
    word_set = set([ word[:i] + char_to_replace_by + word[i+1:] for char_to_replace_by in alphabets for i in range(len(word)) ])
    word_list = list(word_set.difference(set([word])))
    return word_list
    

In [14]:
def switch_letter(word):
    """
    Given a word, it returns all the possible strings that have two adjacent letters switched.

    Params:
    ----------
    word: str
        The word to process
    
    Returns:
    ----------
    word_list: list
        List of possible strings.
    """

    word_list = list(set([ word[:i] + word[i+1] + word[i] + word[i+2:] for i in range(len(word)-1) if word[i] != word[i+1]]))
    return word_list

In [15]:
# Test the functions
print(f'1. delete_letter("hello") -> \n{delete_letter("hello")}\n')
print(f'2. insert_letter("hello") -> \n{insert_letter("hello")}\n')
print(f'3. replace_letter("hello") -> \n{replace_letter("hello")}\n')
print(f'4. switch_letter("hello") -> \n{switch_letter("hello")}\n')

1. delete_letter("hello") -> 
['ello', 'helo', 'hllo', 'hell']

2. insert_letter("hello") -> 
['hellok', 'hellho', 'hellfo', 'helljo', 'hellov', 'hhello', 'heldlo', 'heollo', 'hzello', 'hefllo', 'hellko', 'thello', 'herllo', 'helloe', 'mhello', 'hewllo', 'whello', 'hecllo', 'helilo', 'helwlo', 'hellso', 'helllo', 'hellqo', 'hellto', 'helolo', 'helluo', 'dhello', 'ihello', 'helelo', 'jhello', 'hexllo', 'heqllo', 'hellon', 'hellwo', 'hxello', 'hellom', 'hcello', 'hehllo', 'heillo', 'helloa', 'henllo', 'heullo', 'hebllo', 'helalo', 'haello', 'hellod', 'helglo', 'hellzo', 'hgello', 'hemllo', 'hellol', 'helqlo', 'hellpo', 'hmello', 'hepllo', 'rhello', 'hellor', 'hellao', 'hdello', 'hellou', 'hellob', 'hbello', 'hellro', 'helflo', 'heltlo', 'helrlo', 'zhello', 'ohello', 'hesllo', 'hnello', 'hjello', 'helclo', 'hpello', 'qhello', 'hellog', 'hekllo', 'helldo', 'hellio', 'helloj', 'helklo', 'nhello', 'helmlo', 'hezllo', 'fhello', 'yhello', 'hoello', 'helulo', 'helloo', 'hsello', 'hellyo', 'hell

### Combining Edits
### One Edit Distance Word Generation
We will create a function to get all the possible edits that is `one edit distance` away from a word. The edits  consist of the replace, insert, delete, and optionally the switch operation.

In [16]:
def edit_one_letter(word, allow_switches=True):
    """
    Params:
    ----------
    word: str
        The string/word used to generate all possible words that are one edit distance away.
    allow_switches: bool
        Flag to consider 'switching adjacent letters in a word' as a string edit operation.

    Returns:
    ----------
    edit_one_set: set
        A set of words with one possible edit.
    """
    
    edit_one_set = set()
    
    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))

    return edit_one_set

In [17]:
# Test the edit_one_letter function
print("All possible strings one edit distance away from 'hi' are:\n",edit_one_letter('at'))

All possible strings one edit distance away from 'hi' are:
 {'lt', 'nat', 'dt', 'atc', 'rat', 'uat', 'aqt', 'ant', 'a', 'pt', 'act', 'af', 'vat', 'atz', 'ats', 'ot', 'st', 'hat', 'et', 'aj', 'fat', 'eat', 'atx', 'bat', 'tat', 'atl', 'ate', 'ato', 'bt', 'aw', 'jt', 'cat', 'atp', 'ht', 'wt', 'atm', 'au', 'oat', 'ayt', 'as', 'atn', 'xt', 'dat', 'ast', 'agt', 'apt', 'atq', 'ak', 'aty', 'ag', 'pat', 'qat', 'gt', 'ah', 'an', 'aet', 'abt', 'atj', 'aot', 'att', 'aat', 'alt', 'atu', 'ae', 'ct', 'ad', 'atd', 'atb', 'adt', 'tt', 'ati', 'sat', 'aut', 'ajt', 'ay', 'atk', 'zt', 'ac', 'ar', 'azt', 'kt', 'art', 'ath', 'jat', 'atv', 'ta', 'av', 'awt', 'vt', 'gat', 'ao', 'al', 'aq', 'aft', 'ut', 'ap', 'amt', 'yat', 'xat', 'kat', 'mt', 'ft', 'aht', 'it', 'rt', 'qt', 'mat', 'atf', 'wat', 'ait', 'axt', 'ab', 'atw', 'az', 'ata', 'zat', 'ax', 'iat', 'avt', 'atr', 'yt', 'am', 'aa', 'nt', 'atg', 'ai', 'lat', 'akt', 't'}


We will create a function to get all the possible edits that is `one edit distance` away from a word. The edits  consist of the replace, insert, delete, and optionally the switch operation.

### Two Edit Word Distance Generation
We will generalize from above to get two edits on a word. To do so, we get all the possible single edits on a single word and then for each modified word, we would again apply single edits.

In [18]:
def edit_two_letter(word, allow_switches=True):
    """
    Params:
    ----------
    word: str
        The string/word used to generate all possible words that are two edit distance away.
    allow_switches: bool
        Flag to consider 'switching adjacent letters in a word' as a string edit operation.

    Returns:
    ----------
    edit_two_set: set
        A set of words with two edit distance away from the input word.
    """
    edit_two_set = set()
    for w in edit_one_letter(word, allow_switches=allow_switches):
        edit_two_set.update(edit_one_letter(w, allow_switches=allow_switches))
    
    return edit_two_set

In [19]:
# Test the edit_two_letter function
print("Number of all possible strings two edit distance away from 'hi' are:",len(edit_two_letter('at')))
print("The first five strings two edit distance away from 'hi' are: ",sorted(edit_two_letter('at'))[:5])

Number of all possible strings two edit distance away from 'hi' are: 7154
The first five strings two edit distance away from 'hi' are:  ['', 'a', 'aa', 'aaa', 'aaat']


### Implement Suggestions from Input Word

The 'suggestion algorithm' follows below logic: 
* If the word is in the vocabulary, suggest only that word. 
* Otherwise, if there are suggestions from `edit_one_letter` that are in the vocabulary, use only those. 
* Otherwise, if there are suggestions from `edit_two_letters` that are in the vocabulary, use only those. 
* Otherwise, suggest the input word.

**Note: The idea is - words generated from fewer edits are more likely than words with more edits.**

In [20]:
def get_corrections(word, vocab, word_proba, n_suggestions=1):
    """
    Provides top n best spelling suggestions to the input word.

    Params: 
    ----------
    word: str
        The word to find the suggestions for.
    vocab: list
        The vocabulary to use. This is a list of unique words occuring in the corpus being used.
    word_proba: dict
        The word probabilities dictionary, where the keys are the words in the vocabulary of the corpus 
        and values are the probabilities of the words in the corpus
    n_suggestions: int
        The number of best suggestions to provide.
    
    Returns:
    ----------
    best_words: list
        A list of tuples of the top spelling suggestions for the input word and their probability values.

    """
    

    # generate the suggestions as per the logic
    suggestions = (word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letter(word).intersection(vocab) or word

    # create 'best_words' dictionary where keys are the top spelling 
    # suggestions for the input word and values are the probabilities of those words
    best_words = {}

    if isinstance(suggestions, str): suggestions = [suggestions]
    for w in list(suggestions):
        best_words[w] = word_proba.get(w, 0)
    
    best_words = sorted(best_words.items(), key=lambda item: item[1], reverse=True)
    
    return best_words[:n_suggestions]
    


In [21]:
get_corrections('bithdae', vocab, word_proba, n_suggestions=3)

[('birthday', 4.305551469005312e-05), ('birthdate', 1.136029411347048e-07)]

### Minimum Edit distance

Helps to: 
* Evaluate the similarity between two strings. For example: How similar is 'waht' and 'what'.
* Allows to efficiently find the shortest path to go from the word, 'waht' to the word 'what'.

We use **dynamic programming system** to find the minimum number of edits required to convert a string into another string.

$$\text{Initialization}$$

$$\begin{align}
D[0,0] &= 0 \\
D[i,0] &= D[i-1,0] + del\_cost(source[i]) \tag{1}\\
D[0,j] &= D[0,j-1] + ins\_cost(target[j]) \\
\end{align}$$


$$\text{Per Cell Operations}$$
$$\begin{align}
D[i,j] =min
\begin{cases}
D[i-1,j] + del\_cost\\
D[i,j-1] + ins\_cost\\
D[i-1,j-1] + \left\{\begin{matrix}
rep\_cost; & if src[i]\neq tar[j]\\
0 ; & if src[i]=tar[j]
\end{matrix}\right.
\end{cases}
\tag{2}
\end{align}$$

In [22]:
def min_edit_distance(src, dest, ins_cost=1, del_cost=1, rep_cost=2):
    """
    Compute the minimum edits (distance) required to transform the text 'src' to 'dest'.

    Params:
    ----------
    src: str
        Source string/text.
    dest: str
        Target string/text.

    Returns:
    ----------
    D: numpy array/matrix
        The minimum edit distance matrix for 'src' -> 'dest'
    min_edit_dist: int
        The minimum edit distance 
    """
    
    src = '#' + src
    dest = '#' + dest

    row, col = len(src), len(dest)

    # initialize minimum edit distance array
    D = np.zeros((row, col)).astype(int)
    D[0, :] = np.arange(col)
    D[:, 0] = np.arange(row)

    for i in range(1, row):
        for j in range(1, col):
            D[i, j] = min(D[i-1, j] + del_cost, D[i, j-1] + ins_cost, D[i-1, j-1] + (rep_cost if src[i] != dest[j] else 0))
    
    min_edit_dist = D[row-1, col-1]
    
    return D, min_edit_dist

In [23]:
D, min_edit_dist = min_edit_distance('what', 'waht')

In [24]:
# Test Implementation
src =  'play'
dest = 'stay'
D, min_edit_dist = min_edit_distance(src, dest)
print("minimum edits: ",min_edit_dist, "\n")
idx = list('#' + src)
cols = list('#' + dest)
df = pd.DataFrame(D, index=idx, columns=cols)
print(df)

minimum edits:  4 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4


### Test Model
Test is performed on dataset collected from [Norvig's Article](https://norvig.com/spell-correct.html) which uses Roger Mitton's Birkbeck spelling error corpus from the Oxford Text Archive.

In [69]:
with open('./datasets/test_set.txt', 'r') as file:
    data = file.readlines()

count = 0
correct = 0

for line in data:
    correct_word, misspelled_word_list = line.split(':')
    misspelled_word_list_processed = ' '.join(misspelled_word_list.split('\n')).strip().split(' ')
    for misspelled_word in misspelled_word_list_processed:
        count += 1
        if correct_word == get_corrections(misspelled_word, vocab, word_proba, n_suggestions=1)[0][0]: correct += 1

print(f'Spellings being checked for: {count}')
print(f'Test Accuracy: {correct/count * 100:.2f}%')

Spellings being checked for: 270
Test Accuracy: 63.70%
